In [2]:
import pandas as pd
import time
import re
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
from selenium import webdriver

In [4]:
dates = []
score = []
review_title = []
reviews = []
review_response = []

other_score = []
other_score_label = []
other_score_all = []
other_score_label_all = []

review_date_all = []
review_rating_all = []
review_title_all = []
review_text_all = []
review_response_all = []

df_all = []
df_dates = []
df_ratings = []
df_title = []
df_response = []
df_reviews = []

df_all = pd.DataFrame()
df_detail = pd.DataFrame()

path = 'C:/Temp/geckodriver_win64/geckodriver.exe'

driver = webdriver.Firefox(executable_path=path) #크롬드라이버로 크롬창 켜기

url = 'https://www.tripadvisor.com/Hotel_Review-g294197-d12310284-Reviews-Signiel_Seoul-Seoul.html' # 접속할 웹 사이트 주소 
driver.get(url)  
driver.implicitly_wait(10)

driver.find_elements_by_xpath(".//div[contains(@data-test-target, 'expand-review')]")[0].click()
html = driver.page_source
html = bs(html, 'html.parser')

#pagesequence, indicate number of English reviews. 
eng_rev_num = driver.find_elements_by_class_name('POjZy')[2].text #'(182)'
eng_num = int(re.sub(r'[^0-9]', '', eng_rev_num)) #182
NumReviews = round(eng_num, -1) #round-up!
PageSequence = range(0, NumReviews, 10)
#options(max.print=1000000) #if more than 10,000 reviews

## Proceed with data scraping
for i in tqdm(PageSequence):
    if i == 0:
        driver.get(url)
    else:
        time.sleep(2)
    
    driver.find_elements_by_xpath(".//div[contains(@data-test-target, 'expand-review')]")[0].click()
    html = driver.page_source
    html = bs(html, 'html.parser')
    
    dates = html.find_all('span', {'class':'teHYY _R Me S4 H3'})
    review_date_list = [dates[date].text.replace('Date of stay: ', '') for date in range(len(dates))]
    
    review_title = html.find_all('div', class_= 'KgQgP MC _S b S6 H5 _a')
    review_title_list = [review_title[tit].text for tit in range(len(review_title))]
    
#     reviews = html.find_all('div', class_= '_T FKffI')
    reviews = html.find_all('q', class_= 'QewHA H4 _a')
    review_text_list = [reviews[rev].get_text() for rev in range(len(reviews))]
    
    score = html.find_all('div', class_= 'Hlmiy F1')
    review_rating_list = [int(re.sub(r'[^0-9]', '', score[sc].span['class'][1]).replace('0', '')) for sc in range(len(score))]

    other_score_label = html.find_all('div', class_= 'hemdC S2 H2 WWOoy')
    other_score_label_list = [other_score_label[lab].get_text() for lab in range(len(other_score_label))]
  
    other_score = html.find_all('div', class_= 'hemdC S2 H2 WWOoy')
    other_score_list = [int(re.sub(r'[^0-9]', '', other_score[oth].find_all('span')[1]['class'][1]).replace('0', '')) for oth in range(len(other_score))]
    
#     review_response = html.find_all('div', class_= 'fIrGe _T')
    review_response = html.find_all('span', class_= 'MInAm _a')
    review_response_list = [review_response[res].get_text() for res in range(len(review_response))]

    
    #integrate all data and reviews
    review_date_all.append(review_date_list)
    review_rating_all.append(review_rating_list)
    review_text_all.append(review_text_list)
    review_response_all.append(review_response_list)
    review_title_all.append(review_title_list)
    
    #integrate detailed score labels
    other_score_all.append(other_score_list)
    other_score_label_all.append(other_score_label_list)
    driver.find_elements_by_css_selector(".ui_button.nav.next.primary")[0].click()
    time.sleep(3)
          

path = "C:/Users/user/Desktop/Korean Hotels/"
name = "SignielSeoul"

## Reviews in txt file
df_reviews = pd.DataFrame(sum(review_text_all, []))
save2 = path+name+"-Reviews.txt"
# df_reviews.to_csv(save2)

## Export DATES to csv file
df_dates = pd.DataFrame(sum(review_date_all, []), columns=['Dates'])
save3 = path+name+"-Dates.csv"
# df_dates.to_csv(save3)

## Export ALL to csv file
df_all['Title'] = sum(review_title_all, [])
df_all['Dates'] = sum(review_date_all, [])
df_all['Rating'] = sum(review_rating_all, [])
df_all['Review'] = sum(review_text_all, [])
save4 = path+name+"-All.csv"
# df_all.to_csv(save4)

## Detailed label
df_detail['Attribute'] = sum(other_score_all, [])
df_detail['Rating'] = sum(other_score_label_all, [])
save5 = path+name+"-detail.csv"
# df_detail.to_csv(save5)

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [02:06<00:00,  7.03s/it]
